In [14]:
import numpy as np
import ast
import pytz
import math
import itertools
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_selection import r_regression 
np.random.seed(1337)
df = pd.read_csv('top10K-spotify.csv',index_col=0)
df["type"].value_counts()
df=df.drop(columns=["index","city_1","district_1","district_2","city_3","district_3" ])
df = df[df['city_2'].notna()]
df = df[df['country'].notna()]
df.rename(columns = {'artist':'name'}, inplace = True)

df2=pd.read_csv('data_w_spotify.csv',index_col=None)
df2=df2.drop(columns=["isdone","id"])
df2['genres'] = df2['genres'].fillna('')

df3=df.merge(df2,on='name')

df3['genres']= df3['genres'].str.strip('[]').str.replace("'","").str.split('\s*,\s*')

def create_soup(x):
    return ' '.join(x['genres']) + ' '+ x['city_2'] + ' ' + x['country']+' ' + x['gender']+ ' ' + x['type']

df3['soup'] = df3.apply(create_soup, axis=1)
print(df3.shape,df3.columns)

dict_names = dict(zip(df3.name, df3.spotifyid))

df = df.reset_index()
indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()



def get_recommendations(name,sim,reverse):
    print(sim[0:20])
    # print(name,sim)
    indices = pd.Series(df3.index, index=df3['name']).drop_duplicates()
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

     # Get the movie indices
    indices = [i[0] for i in sim_scores]
    scores = [round(i[1]*100, 2) for i in sim_scores]

    # Return the top 10 most similar movies
     
    top_10=df3['name'].iloc[indices]
    spot_top10=[]
    for item in top_10:
        spot_top10.append(dict_names[str(item)])

    artist=dict_names[str(df3['name'].iloc[idx])]
    
    wantedDict = {
        'mainId': artist,
        'top10': spot_top10,
        'scores': scores
    }
    return wantedDict




(1416, 11) Index(['name', 'gender', 'age', 'type', 'country', 'city_2', 'spotifyid',
       'genres', 'popularity', 'followers', 'soup'],
      dtype='object')


In [15]:
def CVectorizer():
    count = CountVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def CosSim(x):
    return (cosine_similarity(x,x),True)

In [16]:
def TfidVectorizer():
    count = TfidfVectorizer()
    count_matrix = count.fit_transform(df3['soup'])
    return count_matrix

def EuDSim(x):
    return (euclidean_distances(x,), False)

In [17]:
#matrix=TfidVectorizer()
matrix=CVectorizer()
sim,reverse = EuDSim(matrix)
#im,reverse = CosSim(matrix)

get_recommendations("Coldplay",sim,reverse)

[[0.         4.79583152 5.         ... 5.83095189 5.29150262 5.65685425]
 [4.79583152 0.         4.47213595 ... 5.19615242 4.35889894 5.19615242]
 [5.         4.47213595 0.         ... 4.12310563 3.60555128 3.60555128]
 ...
 [5.56776436 5.29150262 2.44948974 ... 4.58257569 4.58257569 4.12310563]
 [4.89897949 2.64575131 4.12310563 ... 4.89897949 4.         4.89897949]
 [5.74456265 5.29150262 3.46410162 ... 4.79583152 4.79583152 4.35889894]]


{'mainId': '4gzpq5DPGxSnKTe4SA8HAU',
 'top10': ['4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '4axSNjwNJoWLJM6HPqxEzz',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '0sYpJ0nCC8AlDrZFeAA7ub',
  '46CitWgnWrvF9t70C2p1Me',
  '0TytASn0kOZkCpjYTrXsBt',
  '7oPftvlwr6VrsViSDV7fJY'],
 'scores': [173.21,
  173.21,
  173.21,
  173.21,
  173.21,
  200.0,
  223.61,
  223.61,
  244.95,
  244.95]}

In [18]:
def Random_tester():
    rand= np.random.randint(0,len(df3),300)
    return rand

sample=Random_tester()
df_sample=pd.DataFrame(sample)
# df_sample.to_csv("random.csv",index=False)

In [49]:
import requests
from time import sleep

apiKey = 'BQDIEuS5XlJmVarqYpvzin2HGiTvVUcF1uLC1885mwG_82Kxooli6hssj1zKUZ91n4b0lefu4kpO_c4xstGnBYp87jPbqfN2h5PIO51OgZS7TdTf3eyDAsq4jxgSKbtwAwWQwq0FL649WVwXR_UU9KZqfehbkhF7_tG4SfwrcIjdoum01nYoKZycDWYXJN8'
BASE_URL = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {token}'.format(token=apiKey)
}


rand_df=pd.read_csv("random.csv",index_col=None)
# print(rand_df.head())
spotifyResultsDf = pd.DataFrame()
i=0
for item,row in rand_df.iterrows():
    id=str(df3['spotifyid'].iloc[row["Num"]])
    topTwentySpotArray = []
    r = requests.get(BASE_URL + 'artists/' + id + '/related-artists', headers=headers)
    rData = r.json()
    for entry in rData['artists']:
        spotId =  entry['id']
        topTwentySpotArray.append(spotId)
        
    if(len(topTwentySpotArray) < 20):
        while len(topTwentySpotArray) < 20:
            topTwentySpotArray.append("")

    print(i)
    i+=1
    numberWanted = str(row['Num'])
    spotifyResultsDf[numberWanted] = topTwentySpotArray
    sleep(0.4)

print(spotifyResultsDf.head())



['1pPWAweVqMBxqTvvWEVtQc', '1ZpLpz4tFdvUocboq1KX3M', '7mnmr8BPHhEEYFwIHlclFt', '0vbfrsbTQsxmBTrvtZTFB4', '0n1kV1G3NQrfuLGJosIGxb', '47YlbKJ9aZM3HVa0lUXHZ0', '3yeIPSXgA3NRNwNaHy4EZW', '2XGaD0VuieQxr4zvfjDzmC', '7b4qgFeygnbXxcH3cvKl1D', '3g5QiiBmEWt5a3WaW1zArB', '4JShzqEj36XDRzWrQnou74', '4P5ooDKThRs2cx6MZ7cpFg', '74pYbw2hCJjXyYitTfyT9F', '4eZg2h2Qez5XuD6PQAqGie', '62TYO8MQIPGNVne59x3AqS', '5PKNAQEEXyfYEmteXrSCGI', '3XoTY8az7uO60cs72LN1K6', '7f9f5zKkxu7tvX5uj7lVwP', '1OJvqVmekd5OPxlTeHmlBl', '0zw17mk0oEnrv4STjNgxsr']
['2VAnyOxzJuSAj7XIuEOT38', '7A0awCXkE1FtSU8B0qwOJQ', '1O10apSOoAPjOu6UhUNmeI', '4VNQWV2y1E97Eqo2D5UTjx', '61lyPtntblHJvA7FMMhi7E', '4csQIMQm6vI2A2SCVDuM2z', '1lc8mnyGrCLtPhCoWjRxjM', '6nxWCVXbOlEVRexSbLsTer', '0GG2cWaonE4JPrjcCCQ1EG', '5vfEaoOBcK0Lzr07WN8KaK', '4VivsO1n4n2Mi2Btyb5gfL', '4qobOrZpdUri80gScwsHfs', '5Q81rlcTFh3k6DQJXPdsot', '5ITI6SEoUZMIXXkzCfr4oE', '2FwJwEswyIUAljqgjNSHgP', '3cIXmCH7iNcslTbwrwS7zy', '3fmMaLC5jjf2N4EC2kTx0u', '5EehXjjMktLuJmbRsM7YfB', '1KpCi9BOf

KeyboardInterrupt: 